In [1]:
import pandas as pd
import matplotlib as mp

%matplotlib inline

In [ ]:
#Algunos datos de utilidad según conclusiones del TP1

#En qué horarios se realizan más subastas?
# Teniendo en cuenta solo los días hábiles
# 5~10    : < 200k
# 4,11~20 : 200k~300k
# 20~0   : 300k ~ 400k
# 1~2    : > 400k

# :. entre las 20~2 hs se realizan la mayor cantidad de subastas con picos entre 1~2 hs.

In [ ]:
# Depende un evento o una conversión de un usuario del día de la semana? Y de la hora?

#La cantidad de eventos se mantine constantes, excepto en los días M y X, donde encontramos un pico.
#Teniendo en cuenta la hora, la mayor cantidad de eventos se produce durante la tarde noche, inicio de la madrugada:
#Pico(+175k):    [0,1]
#Alta(150k~100k):(1,4)U[15,0)
#Media(100k~50k): [4,15)
#Baja(0~25k):(7,9)
#:. La mayor cantidad de eventos sucede durante el rango 15~1(tarde noche)

#INSTALACIONES
# La mayor cantidad se realiza los días M y X
# Los días M y X se duplica la cantidad de instalaciones aproximadamente(350~400)vs(750~800)
# Respecto del horario: Comportamiento similar a los eventos; la mayor cantidad se registran durante la tarde~noche(13~3)
# Afecta la presencia de wifi a la cantidad de instalaciones?
#El 80% de las instalaciones se lleva a cabo con dispositivos usando wifi.
#Cualés son las app mas populares?
# app_id | users | events
# 66       70k      325k
# 63       18k      181k 
# 145      17.5k    252k
# 64       17k      259k
# 103       15k     137k

In [4]:
auctions = pd.read_csv('../7506-tp2/data/auctions.csv.gzip',compression='gzip',\
                        usecols=['date','device_id','platform','ref_type_id','source_id'],\
                        dtype={'device_id':'int64','platform':'category','ref_type_id':'category','source_id':'category'},\
                        parse_dates=['date'])

In [5]:
auctions.head()

,date,device_id,platform,ref_type_id,source_id
0,2019-03-11 14:18:33.290763,6059599345986491085,1,1,0
1,2019-03-11 14:18:34.440157,1115994996230693426,1,1,0
2,2019-03-11 14:18:35.862360,7463856250762200923,1,1,0
3,2019-03-11 14:18:36.167163,7829815350267792040,1,1,0
4,2019-03-11 14:18:37.728590,1448534231953777480,1,1,0
